In [1]:
import pickle;
import numpy as np;
from numpy.linalg import pinv;
from numpy.polynomial.legendre import legvander;
import tensorflow as tf

import control

In [2]:
%matplotlib inline

In [3]:
import cvxopt;
import cvxpy;

def test_synthesis(A,B):
    sf = 10.0
    # Problem data.

    N_u = B.shape[1]
    N_x = A.shape[0]
    Cv =np.eye(N_u)
    Avbar = cvxpy.Variable(rows=N_u,cols=N_u);
    Bvbar = cvxpy.Variable(rows=N_u,cols=N_x);

    P1 = cvxpy.Semidef(A.shape[0]);#,cols=A.shape[0]); # 
    P2 = cvxpy.Semidef(N_u);#,cols=N_u);

    Zeros = cvxpy.Variable(rows=N_x+N_u+N_x+N_u,cols=N_x+N_u+N_x+N_u); 
    
    # Construct the problem.

    objective = cvxpy.Minimize(cvxpy.norm1(Avbar)+cvxpy.norm1(Bvbar));#+cvxpy.norm2(Bv))
    #PMatrix = cvxpy.vstack([cvxpy.hstack([P1,np.zeros(N_x,N_u)]), cvxpy.hstack([np.zeros(( N_u,N_x )),P2]) ]);
    #QMatrix = cvxpy.vstack([cvxpy.hstack([A,np.matmul(B,Cv)]), cvxpy.hstack([Bv,Av]) ]);
    #SystemMatrix = cvxpy.vstack([cvxpy.hstack([PMatrix,QMatrix]),cvxpy.hstack([QMatrix,PMatrix])])
    SystemMatrix1 = cvxpy.hstack([P1, np.zeros((N_x,N_u)), A.T*P1, Bvbar.T]);
    SystemMatrix2 = cvxpy.hstack([np.zeros(( N_u,N_x )),P2,np.matmul(Cv.T,B.T)*P1,Avbar]);
    SystemMatrix3 = cvxpy.hstack([P1*A, P1*np.matmul(B,Cv), P1, np.zeros((N_x,N_u))]);
    SystemMatrix4 = cvxpy.hstack([Bvbar, Avbar, np.zeros((N_u,N_x)), P2]); 
    SystemMatrix = cvxpy.vstack( [SystemMatrix1,SystemMatrix2,SystemMatrix3,SystemMatrix4]);
    constraints = [Zeros==1e-10, Zeros<<SystemMatrix,0<P1,0<P2]; 

    prob = cvxpy.Problem(objective, constraints)

# The optimal objective is returned by prob.solve().
    result = prob.solve(verbose=True,solver='SCS')
    
    return result, Avbar.value, Bvbar.value, Cv, P1.value, P2.value,SystemMatrix.value
# The optimal value for x is stored in x.value.
# The optimal Lagrange multiplier for a constraint
# is stored in constraint.dual_value.
#print constraints[0].dual_value

In [4]:
with_control = 1;

sess = tf.InteractiveSession();

saver = tf.train.import_meta_graph('MD.pickle.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'));

psiyp = tf.get_collection('psiyp')[0];
psiyf = tf.get_collection('psiyf')[0];

yp_feed = tf.get_collection('yp_feed')[0];
yf_feed = tf.get_collection('yf_feed')[0];

psiu = tf.get_collection('psiu')[0];
u_control = tf.get_collection('u_control')[0];

Kx = tf.get_collection('Kx')[0];
Ku = tf.get_collection('Ku')[0];

Kx_num = sess.run(Kx);
Ku_num = sess.run(Ku);
A = np.transpose(Kx_num); # Kx_num and Ku_num were defined using row multi. 
B = np.transpose(Ku_num);
C = np.eye(A.shape[0]);
D = np.zeros((A.shape[0],B.shape[1]));
sys = control.ss(A,B,C,D);
print control.ctrb(A,B)


print np.linalg.matrix_rank(control.ctrb(A,B))
print control.ctrb(A,B).shape
print yp_feed.get_shape



[[ -3.54115993e-01   1.44314335e-03   2.68641859e-03 ...,   4.92665321e-02
   -6.24581464e-02  -1.86721254e-02]
 [ -1.42896399e-01   1.28417555e-03   3.59639269e-03 ...,   4.92659174e-02
   -5.56936674e-02  -1.66286025e-02]
 [ -2.29580954e-01   1.55658019e-03   4.76636877e-03 ...,   4.77671660e-02
   -6.36607036e-02  -1.65715721e-02]
 ..., 
 [ -3.58891994e-01   2.76641134e-04   5.79358777e-04 ...,  -1.42584312e+00
    1.82766452e-01  -1.12962329e+00]
 [ -4.79598522e-01  -6.31193805e-04  -1.83179113e-03 ...,  -1.38961524e-01
    1.27738461e-01  -8.94841645e-03]
 [  7.23206922e-02   1.44754949e-05   5.42946858e-04 ...,   3.26411687e-02
    1.18796438e-01  -1.37587130e-01]]
13
(13, 104)
<bound method Tensor.get_shape of <tf.Tensor 'Placeholder:0' shape=(?, 10) dtype=float32>>


In [5]:
opt_outcome,Avbar_value,Bvbar_value,Cv,P1v,P2v,Sv = test_synthesis(A,B);




----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 8587, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 2227, constraints m = 3363
Cones:	primal zero / dual free vars: 1764
	linear vars: 569
	sd vars: 1030, sd blks: 3
Setup time: 4.96e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.10e-03  4.14e+00  3.62e-02 -3.76e-02  2.95e-13  9.20e-18  6.99e-03 
    20| 2.87e-08  5.43e-05  6.50e-08  6.50e-08  8.44e-13  1.68e-18  1.72e-02 
--------------------------------------------------------------------------

In [6]:
Av = np.matmul(np.linalg.inv(P2v), Avbar_value);
Bv = np.matmul(np.linalg.inv(P2v),Bvbar_value);
print Av.shape
print Bv.shape

(8, 8)
(8, 13)


In [7]:
r1 = np.hstack([A,B])
r2 = np.hstack([Bv,Av]);
cl_mat = np.vstack([r1,r2]);
np.linalg.eig(cl_mat)

(array([-0.60200180+0.j        ,  0.25622656+0.61780023j,
         0.25622656-0.61780023j,  0.98399043+0.00675687j,
         0.98399043-0.00675687j,  0.81839011+0.j        ,
         0.80394073+0.j        ,  0.74603600+0.j        ,
         0.51896008+0.16602487j,  0.51896008-0.16602487j,
        -0.31354066+0.j        ,  0.01595258+0.3321684j ,
         0.01595258-0.3321684j , -0.00371837+0.00790878j,
        -0.00371837-0.00790878j,  0.01712040+0.00932031j,
         0.01712040-0.00932031j,  0.00500751+0.j        ,
         0.01971780+0.j        ,  0.01973985+0.j        ,  0.01982622+0.j        ]),
 matrix([[  8.76555217e-02 +0.00000000e+00j,
           -7.84918615e-03 +3.23085895e-01j,
           -7.84918615e-03 -3.23085895e-01j,
            7.07670232e-04 -3.37954145e-03j,
            7.07670232e-04 +3.37954145e-03j,
           -1.18692643e-04 +0.00000000e+00j,
            3.75060959e-02 +0.00000000e+00j,
            1.37555534e-01 +0.00000000e+00j,
           -5.36701315e-02 -6.425

In [8]:
import scipy.io
import pickle 
scipy.io.savemat('h2synthesis/controller_matrices.mat',mdict={'Av':Av,'Bv':Bv,'Cv':Cv}) #Av,Bv,Cv
file_obj = file('h2synthesis/controller_matrices.pickle','wb');
pickle.dump([Av,Bv,Cv],file_obj,);
file_obj.close();

In [9]:
scipy.io.savemat('h2synthesis/state_space_model.mat',mdict={'A':A,'B':B,'C':C}) #Av,Bv,Cv
file_obj = file('h2synthesis/controller_matrices.pickle','wb');
pickle.dump([A,B,C],file_obj,);
file_obj.close();